<a href="https://colab.research.google.com/github/microprediction/pitllm/blob/main/ChronoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers[torch] accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()


In [12]:
!pip install --upgrade transformers

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoConfig
import torch
from huggingface_hub import HfApi

api = HfApi()
# List all models in the manelalab namespace containing "chrono-gpt"
models = api.list_models(author="manelalab")
chronogpt_models = [m.modelId for m in models if "chrono-gpt" in m.modelId]
print("Available ChronoGPT versions:\n", "\n".join(chronogpt_models))

Available ChronoGPT versions:
 manelalab/chrono-gpt-v1-19991231
manelalab/chrono-gpt-v1-20001231
manelalab/chrono-gpt-v1-20011231
manelalab/chrono-gpt-v1-20021231
manelalab/chrono-gpt-v1-20031231
manelalab/chrono-gpt-v1-20041231
manelalab/chrono-gpt-v1-20051231
manelalab/chrono-gpt-v1-20061231
manelalab/chrono-gpt-v1-20071231
manelalab/chrono-gpt-v1-20081231
manelalab/chrono-gpt-v1-20091231
manelalab/chrono-gpt-v1-20101231
manelalab/chrono-gpt-v1-20111231
manelalab/chrono-gpt-v1-20121231
manelalab/chrono-gpt-v1-20131231
manelalab/chrono-gpt-v1-20141231
manelalab/chrono-gpt-v1-20151231
manelalab/chrono-gpt-v1-20161231
manelalab/chrono-gpt-v1-20171231
manelalab/chrono-gpt-v1-20181231
manelalab/chrono-gpt-v1-20191231
manelalab/chrono-gpt-v1-20201231
manelalab/chrono-gpt-v1-20211231
manelalab/chrono-gpt-v1-20221231
manelalab/chrono-gpt-v1-20231231
manelalab/chrono-gpt-v1-20241231


In [16]:
from huggingface_hub import hf_hub_download
import importlib.util, sys

repo_id = "manelalab/chrono-gpt-v1-20231231"  # pick your cutoff

# 1) Download the file from the repo ✨
path = hf_hub_download(repo_id=repo_id, filename="modeling_chronogpt.py")

# 2) Dynamically import it under a module name you choose (e.g. "chronogpt")
spec = importlib.util.spec_from_file_location("chronogpt", path)
chronogpt = importlib.util.module_from_spec(spec)
sys.modules["chronogpt"] = chronogpt
spec.loader.exec_module(chronogpt)

# 3) Grab the class
ChronoGPT = chronogpt.ChronoGPT


modeling_chronogpt.py:   0%|          | 0.00/8.26k [00:00<?, ?B/s]

In [17]:
model = ChronoGPT.from_pretrained(repo_id).to("cuda")


config.pt:   0%|          | 0.00/924 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

In [23]:
import torch
from tiktoken import get_encoding

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ChronoGPT.from_pretrained(repo_id).to(device)
model.eval()
enc = get_encoding("gpt2")  # GPT‑2 BPE (50304 tokens)

def generate(
    model,
    enc,
    prompt: str,
    max_new_tokens: int = 120,
    temperature: float = 0.8,
    top_p: float = 0.95,
    device: str = "cpu",
):
    """Sample text from ChronoGPT with nucleus sampling."""
    model.eval()
    ids = torch.tensor(enc.encode(prompt), dtype=torch.long)[None].to(device)
    eot_id = enc.eot_token if hasattr(enc, "eot_token") else 50256

    with torch.no_grad():
        for _ in range(max_new_tokens):
            logits, _ = model(ids)
            logits = logits[0, -1] / temperature
            probs = torch.softmax(logits, dim=-1)
            # nucleus (top‑p) sampling
            sorted_probs, sorted_idx = torch.sort(probs, descending=True)
            cumulative = torch.cumsum(sorted_probs, dim=-1)
            cutoff = (cumulative > top_p).nonzero(as_tuple=True)[0][0]
            filtered_idx = sorted_idx[: cutoff + 1]
            next_id = filtered_idx[torch.multinomial(sorted_probs[: cutoff + 1], 1)].item()

            ids = torch.cat([ids, torch.tensor([[next_id]], device=device)], dim=1)
            if next_id == eot_id:
                break
    return enc.decode(ids[0].tolist())

In [24]:
question = "Who was the president of the United States in 2000?"
prompt = f"Q: {question}\nA:"

answer = generate(model, enc, prompt, max_new_tokens=60, temperature=0.7, top_p=0.9, device=device)
print("Q:", question)
print("A:", answer.split("A:")[-1].strip())

Q: Who was the president of the United States in 2000?
A: President Obama
President Obama was the first person to lead the United States in the world in 2000.
President Obama was the first person to lead the United States in the world in 2000.
President Obama was the first person to lead the United States in the world in 2000.
The United States
